In [8]:
import pandas as pd
import cv2,os,glob
import numpy as np

In [9]:
data = pd.read_csv("data/gicsd_labels.csv")

# removing spaces for easy usage
data.columns = ['IMAGE_FILENAME', 'LABEL']

### data is imbalanced

In [16]:
data.LABEL.value_counts()

 FULL_VISIBILITY        646
 PARTIAL_VISIBILITY     123
 NO_VISIBILITY           31
Name: LABEL, dtype: int64

In [17]:
files = glob.glob("data/images/*")

### seggregating images to different folder

In [18]:
data.set_index('IMAGE_FILENAME',drop=True,inplace=True)

In [19]:
os.makedirs('keras_data/FULL_VISIBILITY',exist_ok=True)
os.makedirs('keras_data/PARTIAL_VISIBILITY',exist_ok=True)
os.makedirs('keras_data/NO_VISIBILITY',exist_ok=True)
for f in files:
    filename = os.path.basename(f)
    label = data.loc[filename]['LABEL'].strip()
    os.rename(f,'keras_data/'+label+'/'+filename)
    
    
    

### trying to denoise image

In [20]:
im = cv2.imread('keras_data/FULL_VISIBILITY/GICSD_10_0_13.png')

dst = cv2.fastNlMeansDenoisingColored(im,None,10,10,7,21)


op =np.concatenate((im,dst),axis=1)
cv2.imshow("preview",op)
k = cv2.waitKey(0)
cv2.destroyAllWindows()


### using sift features for classification

In [11]:
data.LABEL = pd.Categorical(data.LABEL)
labels = data.LABEL.cat.codes


In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
features = []
for row in data.iterrows():
    img = cv2.imread('data/images/'+row[0])
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    kp = sift.detect(gray,None)
    features.append(kp)

In [ ]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')
clf.fit(features, labels)

#### modeling(keras models)

In [1]:
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras_preprocessing.image import image_data_generator
from datetime import datetime
from keras.callbacks import ModelCheckpoint,TensorBoard

Using TensorFlow backend.


In [13]:
### preprocessing function
### this can be used to preprocess color images to gray
 cnvt2gray = lambda x : cv2.cvtColor(x,cv2.COLOR_RGB2GRAY)

In [2]:

# base_model = InceptionV3(weights='imagenet', include_top=False)

base_model = ResNet50(weights='imagenet', include_top=False)
train_datagen = image_data_generator.ImageDataGenerator(horizontal_flip=True,vertical_flip=True,shear_range=.5\
                                                        ,rotation_range=30,validation_split=.3)#,preprocessing_function=cnvt2gray)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/home/prateek/.virtualenvs/cv_p36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [9]:
train_generator = train_datagen.flow_from_directory(
    directory="keras_data/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset = "training"
)
validation = train_datagen.flow_from_directory(
    directory="keras_data/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset = "validation"
)


Found 562 images belonging to 3 classes.
Found 238 images belonging to 3 classes.


In [10]:
# add a global spatial average pooling layer
# create the base pre-trained model

x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)

# and a logistic layer -- let's say we have 3 classes
predictions = Dense(3, activation='softmax')(x)
    # this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 pool1_pad
6 max_pooling2d_1
7 res2a_branch2a
8 bn2a_branch2a
9 activation_2
10 res2a_branch2b
11 bn2a_branch2b
12 activation_3
13 res2a_branch2c
14 res2a_branch1
15 bn2a_branch2c
16 bn2a_branch1
17 add_1
18 activation_4
19 res2b_branch2a
20 bn2b_branch2a
21 activation_5
22 res2b_branch2b
23 bn2b_branch2b
24 activation_6
25 res2b_branch2c
26 bn2b_branch2c
27 add_2
28 activation_7
29 res2c_branch2a
30 bn2c_branch2a
31 activation_8
32 res2c_branch2b
33 bn2c_branch2b
34 activation_9
35 res2c_branch2c
36 bn2c_branch2c
37 add_3
38 activation_10
39 res3a_branch2a
40 bn3a_branch2a
41 activation_11
42 res3a_branch2b
43 bn3a_branch2b
44 activation_12
45 res3a_branch2c
46 res3a_branch1
47 bn3a_branch2c
48 bn3a_branch1
49 add_4
50 activation_13
51 res3b_branch2a
52 bn3b_branch2a
53 activation_14
54 res3b_branch2b
55 bn3b_branch2b
56 activation_15
57 res3b_branch2c
58 bn3b_branch2c
59 add_5
60 activation_16
61 res3c_branch2a
62 bn3c_branch2a

In [11]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:161]:
    layer.trainable = False
for layer in model.layers[161:]:
    layer.trainable = True

In [12]:
!mkdir snapshots

mkdir: cannot create directory ‘snapshots’: File exists


In [13]:
from keras.optimizers import SGD
# This assumes that your machine has 8 available GPUs.
# parallel_model = multi_gpu_model(model, gpus=3)
model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir)

checkpoints = [ModelCheckpoint("snapshots/checkpoint-{epoch}.h5", monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1),tensorboard_callback ]


# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(train_generator,epochs=20,steps_per_epoch=30,validation_data=validation,shuffle=True,callbacks=checkpoints)

Epoch 1/20
30/30 [==============================] - 62s 2s/step - loss: 0.6226 - accuracy: 0.7822 - val_loss: 0.8684 - val_accuracy: 0.8109

Epoch 2/20
30/30 [==============================] - 60s 2s/step - loss: 0.4701 - accuracy: 0.8144 - val_loss: 1.0159 - val_accuracy: 0.8109
Epoch 3/20
30/30 [==============================] - 60s 2s/step - loss: 0.4525 - accuracy: 0.8133 - val_loss: 0.7752 - val_accuracy: 0.8109
Epoch 4/20
30/30 [==============================] - 61s 2s/step - loss: 0.3822 - accuracy: 0.8277 - val_loss: 0.5118 - val_accuracy: 0.8109
Epoch 5/20
30/30 [==============================] - 60s 2s/step - loss: 0.4124 - accuracy: 0.8251 - val_loss: 0.1261 - val_accuracy: 0.7605
Epoch 6/20
30/30 [==============================] - 60s 2s/step - loss: 0.3532 - accuracy: 0.8412 - val_loss: 0.8118 - val_accuracy: 0.4286
Epoch 7/20
30/30 [==============================] - 61s 2s/step - loss: 0.3726 - accuracy: 0.8499 - val_loss: 0.4281 - val_accuracy: 0.8067
Epoch 8/20
30/30 [=